More complex format
===================

* xml
* html

XML
===
    * data transfer
    * nested elements
    * document validation
    * parsers in most languages
    * focus on bussines
    * 

```xml
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
```

Html
====

```html
<li class="link">
    <div class="article" data-id="4789115">
        <a href="#4789115">
            <span>85</span>
            <span class="button button submit">wykop</span>
        </a>
    </div>

    <div class="media-content m-reset-float ">
        <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/">
            <img src="https://www.wykop.pl/cdn/c3397993/link_MwUxfsmJdpfvuiy0ICl1l0Yw35VeWgBc,w207h139.jpg" alt="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">
        </a>
    </div>
    <div class="lcontrast m-reset-margin">
        <h2>
            <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                rel="nofollow noopener noreferrer" title="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">W
                jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.</a>
        </h2>


        <div class="description">
            <p class="text">
                <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                    title="">
                    Tragiczna relacja pracownika, który przez lata obserwował jak serwis promujący wolność słowa
                    zamienił się w ideologiczny rynsztok prześladujący pracowników o zróżnicowanym światopoglądzie.
                </a>
            </p>
        </div>
    </div>
</li>
```

In [1]:
!pip install bs4

     |████████████████████████████████| 112kB 661kB/s eta 0:00:01
    Running setup.py install for bs4 ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
sample_xml = '''
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
'''

In [9]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain

In [15]:
!pip install lxml

  Using cached https://files.pythonhosted.org/packages/39/2b/0a66d5436f237aff76b91e68b4d8c041d145ad0a2cdeefe2c42f76ba2857/lxml-4.5.0.tar.gz
    Running setup.py install for lxml ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
loaded = bs(sample_xml, 'lxml')

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?

In [3]:
urls = [
    "https://www.wykop.pl/strona/{}".format(page)
    for page in range(1, 10, 1)
]

In [4]:
import collections
from pprint import pprint as pp

Wykop = collections.namedtuple('Wykop', ['count', 'title', 'description', 'tags'])

def clear_text(text):
    return text.rstrip().replace("\n", '').replace("\t", '')

def grab_wykop_item_data(element):
    try:
        tags = map(lambda x: x.text, element.select('.article>div a.tag'))
        tags = filter(lambda x: x.startswith('#'), tags)
        
        return Wykop(
            int(element.select('.article>div a>span:first-child')[0].text),
            element.select('.article>div h2 a')[0]['title'],
            clear_text(element.select('.article>div .description')[0].text),
            list(tags))
    except Exception as e:
        return None

In [5]:
pages = (requests.get(url) for url in urls)
contents = (page.content for page in pages)
soups = (bs(content, 'html.parser') for content in contents)
lists_of_items = (soup.select('#itemsStream li.iC') for soup in soups)
all_items_together = chain.from_iterable(lists_of_items)
extracted = (grab_wykop_item_data(item) for item in all_items_together)
valid = (wykop for wykop in extracted if wykop is not None)

In [6]:
import itertools
first5 = [next(valid) for x in range(4)]

In [7]:
list(first5)

[Wykop(count=288, title='Rolnicy powracają do starej metody zwalczania szkodników przez sadzenie kwiatów', description='...zamiast używania chemikaliów. Użycie pestycydów okazuje się mieć negatywny wpływ na środowisko. Wielu rolników powraca więc obecnie do starożytnych, przyjaznych dla środowiska metod', tags=['#rolnictwo', '#ekologia', '#srodowisko', '#natura', '#zdrowie', '#polska']),
 Wykop(count=240, title='Specjalny szpital w Wuhan już na ukończeniu! Mają rozmach...', description='Video z dzisiaj. Najbardziej aktualne jakie udało mi się znaleźć. Mimo że jest to tymczasowa budowla to tempo prac i skala projektu naprawdę robią wrażenie. Przewidywane otwarcie placówki 3.02.2020.', tags=['#swiat', '#ciekawostki', '#chiny', '#koronawirus', '#budownictwo', '#wuhan']),
 Wykop(count=934, title='Kushner: Palestyńczycy nie potrafili nic zrobić ze swoją żałosną egzystencją', description='Tak przedstawiciel "rasy panów" i autor planu kolejnego rozbioru Palestyny ubliża ofiarom zbrodni Izrael

In [206]:
tags = (wykop.tags for wykop in first5)

In [207]:
all_tags = list(chain.from_iterable(tags))

In [208]:
counter = {}

for tag in all_tags:
    if tag not in counter.keys():
        counter[tag] = 1
    else:
        counter[tag] = counter[tag] + 1

In [209]:
counter

{'#swiat': 1,
 '#technologia': 2,
 '#facebook': 1,
 '#wojnaidei': 1,
 '#wojnainformacyjna': 1,
 '#kultura': 1,
 '#polska': 3,
 '#drony': 1,
 '#dron': 1,
 '#heheszki': 1,
 '#gdansk': 1,
 '#trojmiasto': 1,
 '#kradziez': 1,
 '#policja': 2,
 '#poznan': 1,
 '#moto': 1,
 '#drogowka': 1}

In [210]:
import operator
list(reversed(sorted(counter.items(), key=lambda item: item[1])))

[('#polska', 3),
 ('#policja', 2),
 ('#technologia', 2),
 ('#drogowka', 1),
 ('#moto', 1),
 ('#poznan', 1),
 ('#kradziez', 1),
 ('#trojmiasto', 1),
 ('#gdansk', 1),
 ('#heheszki', 1),
 ('#dron', 1),
 ('#drony', 1),
 ('#kultura', 1),
 ('#wojnainformacyjna', 1),
 ('#wojnaidei', 1),
 ('#facebook', 1),
 ('#swiat', 1)]

In [217]:
from collections import Counter
c = Counter(all_tags)
c.most_common()

[('#polska', 3),
 ('#technologia', 2),
 ('#policja', 2),
 ('#swiat', 1),
 ('#facebook', 1),
 ('#wojnaidei', 1),
 ('#wojnainformacyjna', 1),
 ('#kultura', 1),
 ('#drony', 1),
 ('#dron', 1),
 ('#heheszki', 1),
 ('#gdansk', 1),
 ('#trojmiasto', 1),
 ('#kradziez', 1),
 ('#poznan', 1),
 ('#moto', 1),
 ('#drogowka', 1)]

persistance
-------------

In [221]:
import csv
with open('wykop.csv', 'w') as destination_file:
    writer = csv.writer(destination_file, delimiter=';', quotechar='"')
    for row in first5:
        writer.writerow(row)

In [226]:
import json
with open('wykop.data', 'w') as destination_file:
    for row in first5:
        destination_file.write(json.dumps(row._asdict()) + "\n")